In [2]:
import pickle
import pandas as pd


with open ('../../data/whole_train_dataset.pandas', 'rb') as file:
    pd__=pickle.load(file)

In [3]:
pd__

,filename,url,project_id,num_frames,crowd_score,tier1,micro,nano,stalled,vid_id
0,100000.mp4,s3://drivendata-competition-clog-loss/train/10...,M,54,0.000000,True,False,False,0,0
1,100001.mp4,s3://drivendata-competition-clog-loss/train/10...,F,48,0.022769,False,False,False,0,1
2,100002.mp4,s3://drivendata-competition-clog-loss/train/10...,H,122,0.000000,True,False,False,0,2
3,100003.mp4,s3://drivendata-competition-clog-loss/train/10...,E,55,0.000000,True,False,False,0,3
4,100004.mp4,s3://drivendata-competition-clog-loss/train/10...,C,56,0.000000,True,False,False,0,4
...,...,...,...,...,...,...,...,...,...,...
573043,687207.mp4,s3://drivendata-competition-clog-loss/train/68...,G,93,0.000000,True,False,False,0,573043
573044,687208.mp4,s3://drivendata-competition-clog-loss/train/68...,A,60,0.000000,True,False,False,0,573044
573045,687209.mp4,s3://drivendata-competition-clog-loss/train/68...,E,49,0.000000,True,False,False,0,573045
573046,687210.mp4,s3://drivendata-competition-clog-loss/train/68...,D,42,0.151080,False,False,False,0,573046


In [2]:
%%javascript 

IPython.OutputArea.prototype._should_scroll = function(lines) { 
    return false; } 

<IPython.core.display.Javascript object>

In [27]:
# The Code written by Ali Babolhaveji @ 6/7/2020

from torch.utils.data import Dataset
import pandas as pd
import numpy as np
import cv2
import ipyvolume as ipv
from boto3.session import Session
import os
import yaml
import joblib
import pickle

from tqdm.notebook import tqdm

# def extract_data(row):
#     joblib.load()

from sklearn.model_selection import KFold


def genrate_K_folds(dataset_pd , K=2 , Fold =0 ,random_state=None ,shuffle=False):


    kf = KFold(n_splits=K, random_state=random_state, shuffle=shuffle)
    assert Fold < kf.n_splits , f"Fold number should be between [0 and {K-1}] but it is {Fold} "
    for num,(train, val) in enumerate(kf.split(dataset_pd)):
#         print(f"Fold-{num}",'train: %s, val: %s' % (train, val))
        if Fold == num:
            return train, val
    


class ClogLossDataset_from_compressed_data(Dataset):
    def __init__(self, config, split='train', type='train' , draw_3d = False , fold=0):
        self.cfg = config
        self.dataPath = config['dataset']['path']
        self.draw_3d = draw_3d
        self.fold = fold
        self.split = split
        
        with open (os.path.join(self.dataPath , 'flowing_Tensors','flowing_Tensors.pandas'),'rb') as handle:
            self.flowing_Tensors_pd = pickle.load(handle)
        self.flowing_Tensors_pd['folder_name'] = np.tile('flowing_Tensors',len(self.flowing_Tensors_pd))
        
        with open (os.path.join(self.dataPath , 'stall_Tensors','stall_Tensors.pandas'),'rb') as handle:
            self.stall_Tensors_pd = pickle.load(handle)
        self.stall_Tensors_pd['folder_name'] = np.tile('stall_Tensors',len(self.stall_Tensors_pd))
        
        if not os.path.exists(os.path.join(self.dataPath, "temp_balanced_dataset_pd.pandas")):
            self.df_dataset = pd.DataFrame()
    #         self.flowing_Tensors_pd = self.flowing_Tensors_pd.iloc[:100]
            counter = 0   
            len_stall = len(self.stall_Tensors_pd)-1
            # balance data
            for  rowt in self.flowing_Tensors_pd.iterrows():
                indx=rowt[0]
                row =rowt[1]

                self.df_dataset = self.df_dataset.append(row)
                self.df_dataset = self.df_dataset.append(self.stall_Tensors_pd.iloc[counter])
                counter = [0 ,counter+1][counter <len_stall] 

            self.df_dataset = self.df_dataset.reset_index(drop=True)

            with open(os.path.join(self.dataPath, f"temp_balanced_dataset_pd.pandas"),'wb') as handel:
                pickle.dump(self.df_dataset , handel ,protocol=pickle.HIGHEST_PROTOCOL)
                
        else:
            with open (os.path.join(self.dataPath, f"temp_balanced_dataset_pd.pandas"),'rb') as handle:
                self.df_dataset = pickle.load(handle)
                
        train, val = genrate_K_folds(self.df_dataset, K=self.cfg['dataset']['K'], Fold=self.fold)
        
        if self.split == 'train':
            self.df_dataset = self.df_dataset.iloc[train]

        elif self.split == 'val':
            self.df_dataset = self.df_dataset.iloc[val]

            
    #         print(self.df_dataset )
        
    @staticmethod
    def draw_tensor(tensor_img):

        ipv.figure()
        ipv.volshow(tensor_img[..., 0], level=[0.36, 0.55, 1], opacity=[0.11, 0.13, 0.13], level_width=0.05, data_min=0,
                    data_max=1, lighting=True)
        ipv.view(-30, 45)
        ipv.show()

            
            
    def __len__(self):
        return len(self.df_dataset)
    
    def __getitem__(self, index):
        row = self.df_dataset.iloc[index]
        file= os.path.join(self.dataPath,row.folder_name  , row.filename.split('.')[0]+'.lzma')
        data = joblib.load(file)
        tensor_img = data[0]
        meta = data [1]
        
        print(tensor_img.mean())
#         tensor_img = tensor_img - tensor_img.mean()
        tensor_img = tensor_img/255.0
        
        if tensor_img.shape[0]<199:
            print(tensor_img.shape[0])
            tensor_img = np.append(tensor_img , np.zeros((200 - len(tensor_img),150, 150, 3)),axis=0)
        
#         print(tensor_img.mean())
        if self.draw_3d:
            self.draw_tensor(tensor_img)
        meta['tier1']= str(meta['tier1'])
        
        tensor_img = np.moveaxis(tensor_img ,3,0)
        tensor_img = tensor_img.astype(np.float32)

        return tensor_img , meta

    
config = '../script/config.yml'
with open (config , 'rb') as f:
    config = yaml.load(f ,Loader=yaml.FullLoader)


# train_Dataset = ClogLossDataset_from_compressed_data(config , draw_3d=True)
train_Dataset = ClogLossDataset_from_compressed_data(config )

In [19]:
len(train_Dataset)

9000

In [28]:
train_Dataset[0][0].shape

6.676209613869188
47


(3, 200, 150, 150)

In [73]:
for i in tqdm(range(len(train_Dataset))):
    pass
#     print(i)